In [1]:
import os
import sys
import pickle
import time

PROJECT_ROOT = os.path.abspath('..')
sys.path.append(PROJECT_ROOT)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd
import scipy
from scipy import stats
from tqdm import tqdm

from sleeprnn.helpers.reader import load_dataset
from sleeprnn.common import constants, viz, pkeys
from sleeprnn.data import utils, stamp_correction
from sleeprnn.detection.postprocessor import PostProcessor
from sleeprnn.detection.predicted_dataset import PredictedDataset
from sleeprnn.detection.feeder_dataset import FeederDataset
from sleeprnn.detection import det_utils
from figs_thesis import fig_utils

viz.notebook_full_width()

param_filtering_fn = fig_utils.get_filtered_signal_for_event
param_frequency_fn = fig_utils.get_frequency_by_fft
param_amplitude_fn = fig_utils.get_amplitude_event

RESULTS_PATH = os.path.join(PROJECT_ROOT, 'results')
LETTERS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']

/home/ntapia/miniconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
def load_predictions(parts_to_load, dataset):
    pred_objects = []
    for part in parts_to_load:
        filepath = os.path.join(
            RESULTS_PATH, 'predictions_nsrr_ss',
            'ckpt_20210716_from_20210529_thesis_indata_5cv_e1_n2_train_moda_ss_ensemble_to_e1_n2_train_nsrr_ss',
            'v2_time',
            'prediction_part%d.pkl' % part
        )
        with open(filepath, 'rb') as handle:
            pred_object = pickle.load(handle)
        pred_object.set_parent_dataset(dataset)
        pred_objects.append(pred_object)
    return pred_objects

# Load NSRR dataset and pre-computed predicted dataset

In [3]:
parts_to_load = [0]  # 0 to 11

nsrr = load_dataset(constants.NSRR_SS_NAME, load_checkpoint=True)
pred_objects = load_predictions(parts_to_load, nsrr)

Dataset nsrr_ss with 3 patients.
Loading from checkpoint... Loaded
Global STD: None
Dataset nsrr_ss with 11593 patients.


In [4]:
# Filenames of dataset checkpoints
byevent_proba_ckpt_path = os.path.join(
    RESULTS_PATH, 'predictions_nsrr_ss',
    'ckpt_20210716_from_20210529_thesis_indata_5cv_e1_n2_train_moda_ss_ensemble_to_e1_n2_train_nsrr_ss',
    'v2_time',
    'table_byevent_proba.csv'
)

In [6]:
params_load_checkpoint = False

# ############################

if params_load_checkpoint:
    print("Loading from checkpoint")
    table_byevent_proba = pd.read_csv(byevent_proba_ckpt_path)

else:
    # Perform computation and save checkpoint
    table_byevent_proba = {
        'subject_id': [],
        'center_sample': [],
        'prediction_part': [],
        'class': [],
        'probability': [],
        'duration': [], 
        'frequency': [],
        'amplitude_pp': [],
        'amplitude_rms': [],
        'simetry': [],
        'c10_density': [],
        'c20_density': [],
        
    }

    min_n2_minutes = 60
    verbose_min_minutes = False

    start_time = time.time()
    print("Generating table of parameters")
    for part_id, predictions in enumerate(pred_objects):
        print("Processing Part %d / %d" % (part_id + 1, len(pred_objects)))
        # Class 1 spindles (real):
        marks_1_list = predictions.get_stamps(pages_subset='n2')
        proba_1_list = predictions.get_stamps_probabilities(pages_subset='n2')
        # Class 0 "spindles" (false):
        predictions.set_probability_threshold(0.25)
        marks_0_list = predictions.get_stamps(pages_subset='n2')
        proba_0_list = predictions.get_stamps_probabilities(pages_subset='n2')
        for i_subject in tqdm(range(100)):
            subject_id = predictions.all_ids[i_subject]
            
            n2_pages = predictions.data[subject_id]['n2_pages']
            n2_minutes = n2_pages.size * nsrr.original_page_duration / 60
            if n2_minutes < min_n2_minutes:
                if verbose_min_minutes:
                    print("Skipped by N2 minutes: Subject %s with %d N2 minutes" % (subject_id, n2_minutes))
                continue

            marks_1 = marks_1_list[i_subject]
            proba_1 = proba_1_list[i_subject]
            marks_0 = marks_0_list[i_subject]
            proba_0 = proba_0_list[i_subject]

            # Let only those class 0 without intersecting class 1
            if marks_1.size > 0:
                ov_mat = utils.get_overlap_matrix(marks_0, marks_1)
                is_intersecting = ov_mat.sum(axis=1) - 1
                print(marks_1.shape, marks_0.shape, ov_mat.shape, is_intersecting.shape)
            else:
                print("no marks 1, accepting all marks 0", marks_1.shape, marks_0.shape)
            

Generating table of parameters
Processing Part 1 / 1


KeyboardInterrupt: 